In [ ]:
!pip install xarray boto3 rasterio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 60.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 76.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.6/82.6 kB 6.2 MB/s eta 0:00:00


In [ ]:
import subprocess

import boto3
from botocore import UNSIGNED
from botocore.client import Config

import xarray as xr
import numpy as np
from scipy.interpolate import griddata

import rasterio
from rasterio.transform import from_bounds


In [ ]:
import ee

# Trigger the authentication flow.
ee.Authenticate()

PROJECT_ID = 'pyregence-ee'

# Initialize the library.
ee.Initialize(project=PROJECT_ID, opt_url='https://earthengine-highvolume.googleapis.com')

In [ ]:
base_path = f'projects/{PROJECT_ID}/assets/'
# ee_path = 'wrf-data/wrf-data-etrans-sfc'
ee_path = 'wrf-data/relative-humidity'

In [ ]:
# see variable names here: https://dept.atmos.ucla.edu/sites/default/files/alexhall/files/aws_tiers_dirstructure_nov22.pdf
# 'soil_m' # etrans_sfc # sh_sfc
wrf_variable_name = 'rh'

In [ ]:
# original link to download: https://wrf-cmip6-noversioning.s3.amazonaws.com/index.html#downscaled_products/gcm/canesm5_r1i1p2f1_ssp370_bc/postprocess/d02/

# Initialize a session using Amazon S3 (unsigned access)
s3 = boto3.client('s3', config=Config(signature_version=UNSIGNED))

# Define the bucket and file key for etrans_sfc file
bucket_name = 'wrf-cmip6-noversioning'
historical_wrf_variable_folder_path = 'downscaled_products/gcm/canesm5_r1i1p2f1_historical_bc/postprocess/d02'
historical_wrf_variable_file_key = wrf_variable_name+'.daily.canesm5.r1i1p2f1.hist.bias-correct.d02.{}.nc'
hist_years = range(1990, 2014)
gcp_bucket = f'wrf-{wrf_variable_name}'

future_wrf_variable_folder_path = 'downscaled_products/gcm/canesm5_r1i1p2f1_ssp370_bc/postprocess/d02'
future_wrf_variable_file_key = wrf_variable_name + '.daily.canesm5.r1i1p2f1.ssp370.bias-correct.d02.{}.nc'
future_years = range(2014, 2031)

# Define the file path for the coordinate file (wrfinput_d02_coord.nc)
coord_file_name = 'wrfinput_d02_coord.nc'
coord_file_key = f'downscaled_products/wrf_coordinates/{coord_file_name}'


### convert netcdfs (historical)

In [ ]:
# Download the wrfinput_d02_coord.nc file
s3.download_file(bucket_name, coord_file_key, coord_file_name)

In [ ]:
wrfinput = xr.open_dataset(coord_file_name)

In [ ]:
for year in hist_years:
    print(f'download historical {year}')
    f_name = historical_wrf_variable_file_key.format(year)
    # Download the etrans_sfc file
    s3.download_file(bucket_name, f'{historical_wrf_variable_folder_path}/{f_name}', f_name)



download historical 1990
download historical 1991
download historical 1992
download historical 1993
download historical 1994
download historical 1995
download historical 1996
download historical 1997
download historical 1998
download historical 1999
download historical 2000
download historical 2001
download historical 2002
download historical 2003
download historical 2004
download historical 2005
download historical 2006
download historical 2007
download historical 2008
download historical 2009
download historical 2010
download historical 2011
download historical 2012
download historical 2013


In [ ]:
def get_saved_file_name(file_name, soil_m_level=None):
    save_file_name = file_name
    if soil_m_level:
        try:
            soil_m_mapping = {
                1: '5-cm',
                2: '25-cm',
                3: '70-cm',
                4: '150-cm',
            }
            save_file_name += f'_level_{soil_m_mapping[soil_m_level]}'
        except KeyError:
            raise ValueError(f'soil_m_level not supported for {wrf_variable_name}')

    save_file_name += f'.daily.canesm5.r1i1p2f1.{year}.tif'
    return save_file_name

In [ ]:
def process_netcdf(file_name, soil_m_level=None):
    nc_file = xr.open_dataset(file_name)

    # Extract lat2d and lon2d
    lat2d = wrfinput['lat2d'].values
    lon2d = wrfinput['lon2d'].values

    if wrf_variable_name == 'soil_m' and soil_m_level:
        average_wrf_var = nc_file[wrf_variable_name].isel(soil_nz=soil_m_level-1).mean(dim='day').values
    else:
        average_wrf_var = nc_file[wrf_variable_name].mean(dim='day').values

    # Create a regular lon-lat grid for interpolation (note the swapped order)
    lon_interp = np.linspace(np.min(lon2d), np.max(lon2d), average_wrf_var.shape[1])
    lat_interp = np.linspace(np.min(lat2d), np.max(lat2d), average_wrf_var.shape[0])

    # Create a meshgrid for the new grid
    lon_mesh, lat_mesh = np.meshgrid(lon_interp, lat_interp)

    # Flatten the original lat/lon and data values for interpolation
    lat_lon_points = np.array([lat2d.flatten(), lon2d.flatten()]).T
    average_wrf_var_flat = average_wrf_var.flatten()


    # Perform interpolation to the regular grid
    grid_data = griddata(lat_lon_points, average_wrf_var_flat, (lat_mesh, lon_mesh), method='linear')

    # Flip the grid data vertically to correct the upside-down orientation
    grid_data_flipped = np.flipud(grid_data)

    # Save the interpolated data as a GeoTIFF with correct bounds
    transform = from_bounds(np.min(lon_interp), np.min(lat_interp), np.max(lon_interp), np.max(lat_interp), grid_data_flipped.shape[1], grid_data_flipped.shape[0])

    saved_file_name = get_saved_file_name(wrf_variable_name, soil_m_level=soil_m_level)

    with rasterio.open(saved_file_name, 'w', driver='GTiff',
                       height=grid_data_flipped.shape[0], width=grid_data_flipped.shape[1], count=1,
                       dtype=grid_data_flipped.dtype, crs='EPSG:4326', transform=transform) as dst:
        dst.write(grid_data_flipped, 1)
    return saved_file_name


In [ ]:
wrf_variable_name

'rh'

In [ ]:
f_name

'rh.daily.canesm5.r1i1p2f1.hist.bias-correct.d02.2013.nc'

In [ ]:
if wrf_variable_name != 'soil_m':
    soil_m_level = None
else:
    soil_m_level = 4 # soil_m_level index starts at 1 and ends at 4

In [ ]:
soil_m_level

In [ ]:
for year in hist_years:
    print(f'processed historical {year}')
    f_name = historical_wrf_variable_file_key.format(year)
    saved_file_name = process_netcdf(f_name, soil_m_level)


processed historical 1990
processed historical 1991
processed historical 1992
processed historical 1993
processed historical 1994
processed historical 1995
processed historical 1996
processed historical 1997
processed historical 1998
processed historical 1999
processed historical 2000
processed historical 2001
processed historical 2002
processed historical 2003
processed historical 2004
processed historical 2005
processed historical 2006
processed historical 2007
processed historical 2008
processed historical 2009
processed historical 2010
processed historical 2011
processed historical 2012
processed historical 2013


In [ ]:
saved_file_name

'rh.daily.canesm5.r1i1p2f1.2013.tif'

In [ ]:
with rasterio.open(saved_file_name) as dataset:
    # resolution (in degrees)
    resolution_deg = dataset.res

    lat_center = (dataset.bounds.top + dataset.bounds.bottom) / 2

    # Convert resolution in degrees to meters
    resolution_lat_m = resolution_deg[1] * 111320  # Resolution for latitude (constant ~111.32 km per degree)
    resolution_lon_m = resolution_deg[0] * 111320 * np.cos(np.radians(lat_center))  # Resolution for longitude

    print(f"Approximate resolution in meters: {resolution_lon_m:.2f} m x {resolution_lat_m:.2f} m")


Approximate resolution in meters: 13749.25 m x 11562.08 m


### make cogs

In [ ]:
!apt-get install -y gdal-bin python3-gdal

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  python3-numpy
Suggested packages:
  libgdal-grass python-numpy-doc python3-pytest
The following NEW packages will be installed:
  gdal-bin python3-gdal python3-numpy
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 5,055 kB of archives.
After this operation, 25.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 python3-numpy amd64 1:1.21.5-1ubuntu22.04.1 [3,467 kB]
Get:2 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy/main amd64 python3-gdal amd64 3.6.4+dfsg-1~jammy0 [1,027 kB]
Get:3 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy/main amd64 gdal-bin amd64 3.6.4+dfsg-1~jammy0 [561 kB]
Fetched 5,055 kB in 2s (2,169 kB/s)
Selecting previously unselected package python3-numpy.
(Reading database ... 123629 files and directories curren

In [ ]:
!gdalinfo --version


GDAL 3.6.4, released 2023/04/17


In [ ]:
# save cogs
def save_cogs(filename):
    if filename.endswith('.tif'):
        filename = filename[:-4]
    print('filename', filename)
    cog_cmd = f'gdal_translate {filename}.tif {filename}_cog.tif -co TILED=YES -co COPY_SRC_OVERVIEWS=YES -co COMPRESS=LZW'
    print(f"cog_cmd >> : {cog_cmd}")

    result = subprocess.check_output(cog_cmd, shell=True)
    print("result", result)


In [ ]:
soil_m_level

In [ ]:
for year in hist_years:
    print(f'uploaded historical {year}')
    f_name = get_saved_file_name(wrf_variable_name, soil_m_level=soil_m_level)
    # f_name = f'{wrf_variable_name}.daily.canesm5.r1i1p2f1.{year}'
    save_cogs(f_name)

uploaded historical 1990
filename rh.daily.canesm5.r1i1p2f1.1990
cog_cmd >> : gdal_translate rh.daily.canesm5.r1i1p2f1.1990.tif rh.daily.canesm5.r1i1p2f1.1990_cog.tif -co TILED=YES -co COPY_SRC_OVERVIEWS=YES -co COMPRESS=LZW
result b'Input file size is 270, 340\n0...10...20...30...40...50...60...70...80...90...100 - done.\n'
uploaded historical 1991
filename rh.daily.canesm5.r1i1p2f1.1991
cog_cmd >> : gdal_translate rh.daily.canesm5.r1i1p2f1.1991.tif rh.daily.canesm5.r1i1p2f1.1991_cog.tif -co TILED=YES -co COPY_SRC_OVERVIEWS=YES -co COMPRESS=LZW
result b'Input file size is 270, 340\n0...10...20...30...40...50...60...70...80...90...100 - done.\n'
uploaded historical 1992
filename rh.daily.canesm5.r1i1p2f1.1992
cog_cmd >> : gdal_translate rh.daily.canesm5.r1i1p2f1.1992.tif rh.daily.canesm5.r1i1p2f1.1992_cog.tif -co TILED=YES -co COPY_SRC_OVERVIEWS=YES -co COMPRESS=LZW
result b'Input file size is 270, 340\n0...10...20...30...40...50...60...70...80...90...100 - done.\n'
uploaded historical

### upload to cloud

In [ ]:
# create bucket if not exists
REGION = 'us-central1'

from google.cloud import storage
from google.api_core.exceptions import Conflict

def create_bucket_if_not_exists(bucket_name, location=REGION):
    # Initialize a storage client
    storage_client = storage.Client(project=PROJECT_ID)

    # Check if the bucket already exists
    try:
        bucket = storage_client.get_bucket(bucket_name)
        print(f'Bucket {bucket_name} already exists.')
    except Exception as e:
        if isinstance(e, Conflict):
            print(f'Bucket {bucket_name} already exists.')
        else:
            # If the bucket does not exist, create it
            bucket = storage_client.bucket(bucket_name)
            bucket.location = location
            bucket.create()
            print(f'Bucket {bucket_name} created at location {location}.')

create_bucket_if_not_exists(gcp_bucket)


<ipython-input-25-8fcf5dd650a2>:21: DeprecationWarning: Assignment to 'Bucket.location' is deprecated, as it is only valid before the bucket is created. Instead, pass the location to `Bucket.create`.
  bucket.location = location


Bucket wrf-rh created at location us-central1.


In [ ]:
def upload_to_gcp(filename):
    if filename.endswith('.tif'):
        filename = filename[:-4]
    # Copy the file
    cp = f"gsutil cp {filename}_cog.tif gs://{gcp_bucket}/{filename}.tif"
    print(f"cp >> : {cp}")

    result = subprocess.check_output(cp, shell=True)
    print("result", result)

In [ ]:
for year in hist_years:
    print(f'uploaded historical {year}')
    # f_name = f'{wrf_variable_name}.daily.canesm5.r1i1p2f1.{year}'
    f_name = get_saved_file_name(wrf_variable_name, soil_m_level=soil_m_level)
    upload_to_gcp(f_name)


uploaded historical 1990
cp >> : gsutil cp rh.daily.canesm5.r1i1p2f1.1990_cog.tif gs://wrf-rh/rh.daily.canesm5.r1i1p2f1.1990.tif
result b''
uploaded historical 1991
cp >> : gsutil cp rh.daily.canesm5.r1i1p2f1.1991_cog.tif gs://wrf-rh/rh.daily.canesm5.r1i1p2f1.1991.tif
result b''
uploaded historical 1992
cp >> : gsutil cp rh.daily.canesm5.r1i1p2f1.1992_cog.tif gs://wrf-rh/rh.daily.canesm5.r1i1p2f1.1992.tif
result b''
uploaded historical 1993
cp >> : gsutil cp rh.daily.canesm5.r1i1p2f1.1993_cog.tif gs://wrf-rh/rh.daily.canesm5.r1i1p2f1.1993.tif
result b''
uploaded historical 1994
cp >> : gsutil cp rh.daily.canesm5.r1i1p2f1.1994_cog.tif gs://wrf-rh/rh.daily.canesm5.r1i1p2f1.1994.tif
result b''
uploaded historical 1995
cp >> : gsutil cp rh.daily.canesm5.r1i1p2f1.1995_cog.tif gs://wrf-rh/rh.daily.canesm5.r1i1p2f1.1995.tif
result b''
uploaded historical 1996
cp >> : gsutil cp rh.daily.canesm5.r1i1p2f1.1996_cog.tif gs://wrf-rh/rh.daily.canesm5.r1i1p2f1.1996.tif
result b''
uploaded historical 

### upload to ee

In [ ]:
# Set default project
!earthengine set_project {PROJECT_ID}
!gcloud config set project {PROJECT_ID}

Successfully saved project id
Updated property [core/project].


In [ ]:
PROJECT_ID

'pyregence-ee'

In [ ]:
f'{base_path}{ee_path}'

'projects/pyregence-ee/assets/wrf-data/relative-humidity'

In [ ]:
# create collection if not exists
try:
    ee.data.createAsset({'type': 'IMAGE_COLLECTION'}, f'{base_path}{ee_path}', False)
except ee.EEException:
    print('assets already exists')


In [ ]:
for year in hist_years:
    # f_name = f'{wrf_variable_name}.daily.canesm5.r1i1p2f1.{year}'
    f_name = get_saved_file_name(wrf_variable_name, soil_m_level=soil_m_level)
    img = ee.Image.loadGeoTIFF(f'gs://{gcp_bucket}/{f_name}')
    img = img.set(
        'system:time_start', ee.Date(f'{year}-01-01').millis(),
        'system:time_end', ee.Date(f'{year}-12-31').millis(),
        'soil_m_level', soil_m_level,
        'wrf_variable_name', wrf_variable_name,
    )
    export_image = f'projects/{PROJECT_ID}/assets/{ee_path}/{"_".join(f_name.split("."))}'
    print(f'uploading to {export_image}')
    image_task = ee.batch.Export.image.toAsset(
        image=img,
        description=f'{f_name}',
        assetId=export_image,
        region=img.geometry().bounds(),
        scale=5000,
        maxPixels=1e13,
    )

    image_task.start()


uploading to projects/pyregence-ee/assets/wrf-data/relative-humidity/rh_daily_canesm5_r1i1p2f1_1990_tif
uploading to projects/pyregence-ee/assets/wrf-data/relative-humidity/rh_daily_canesm5_r1i1p2f1_1991_tif
uploading to projects/pyregence-ee/assets/wrf-data/relative-humidity/rh_daily_canesm5_r1i1p2f1_1992_tif
uploading to projects/pyregence-ee/assets/wrf-data/relative-humidity/rh_daily_canesm5_r1i1p2f1_1993_tif
uploading to projects/pyregence-ee/assets/wrf-data/relative-humidity/rh_daily_canesm5_r1i1p2f1_1994_tif
uploading to projects/pyregence-ee/assets/wrf-data/relative-humidity/rh_daily_canesm5_r1i1p2f1_1995_tif
uploading to projects/pyregence-ee/assets/wrf-data/relative-humidity/rh_daily_canesm5_r1i1p2f1_1996_tif
uploading to projects/pyregence-ee/assets/wrf-data/relative-humidity/rh_daily_canesm5_r1i1p2f1_1997_tif
uploading to projects/pyregence-ee/assets/wrf-data/relative-humidity/rh_daily_canesm5_r1i1p2f1_1998_tif
uploading to projects/pyregence-ee/assets/wrf-data/relative-humi

### convert netcdfs (future)

In [ ]:
# Initialize a session using Amazon S3 (unsigned access)
s3 = boto3.client('s3', config=Config(signature_version=UNSIGNED))

In [ ]:
for year in future_years:
    print(f'download future {year}')
    f_name = future_wrf_variable_file_key.format(year)
    # Download the etrans_sfc file
    s3.download_file(bucket_name, f'{future_wrf_variable_folder_path}/{f_name}', f_name)



download future 2014
download future 2015
download future 2016
download future 2017
download future 2018
download future 2019
download future 2020
download future 2021
download future 2022
download future 2023
download future 2024
download future 2025
download future 2026
download future 2027
download future 2028
download future 2029
download future 2030


In [ ]:
for year in future_years:
    print(f'processed historical {year}')
    f_name = future_wrf_variable_file_key.format(year)
    saved_file_name = process_netcdf(f_name, soil_m_level)


processed historical 2014
processed historical 2015
processed historical 2016
processed historical 2017
processed historical 2018
processed historical 2019
processed historical 2020
processed historical 2021
processed historical 2022
processed historical 2023
processed historical 2024
processed historical 2025
processed historical 2026
processed historical 2027
processed historical 2028
processed historical 2029
processed historical 2030


### make cogs

In [ ]:
for year in future_years:
    print(f'uploaded historical {year}')
    f_name = get_saved_file_name(wrf_variable_name, soil_m_level=soil_m_level)
    # f_name = f'{wrf_variable_name}.daily.canesm5.r1i1p2f1.{year}'
    save_cogs(f_name)

uploaded historical 2014
filename rh.daily.canesm5.r1i1p2f1.2014
cog_cmd >> : gdal_translate rh.daily.canesm5.r1i1p2f1.2014.tif rh.daily.canesm5.r1i1p2f1.2014_cog.tif -co TILED=YES -co COPY_SRC_OVERVIEWS=YES -co COMPRESS=LZW
result b'Input file size is 270, 340\n0...10...20...30...40...50...60...70...80...90...100 - done.\n'
uploaded historical 2015
filename rh.daily.canesm5.r1i1p2f1.2015
cog_cmd >> : gdal_translate rh.daily.canesm5.r1i1p2f1.2015.tif rh.daily.canesm5.r1i1p2f1.2015_cog.tif -co TILED=YES -co COPY_SRC_OVERVIEWS=YES -co COMPRESS=LZW
result b'Input file size is 270, 340\n0...10...20...30...40...50...60...70...80...90...100 - done.\n'
uploaded historical 2016
filename rh.daily.canesm5.r1i1p2f1.2016
cog_cmd >> : gdal_translate rh.daily.canesm5.r1i1p2f1.2016.tif rh.daily.canesm5.r1i1p2f1.2016_cog.tif -co TILED=YES -co COPY_SRC_OVERVIEWS=YES -co COMPRESS=LZW
result b'Input file size is 270, 340\n0...10...20...30...40...50...60...70...80...90...100 - done.\n'
uploaded historical

### upload to cloud

In [ ]:
for year in future_years:
    print(f'uploaded future {year}')
    # f_name = f'{wrf_variable_name}.daily.canesm5.r1i1p2f1.{year}'
    f_name = get_saved_file_name(wrf_variable_name, soil_m_level=soil_m_level)
    upload_to_gcp(f_name)


uploaded future 2014
cp >> : gsutil cp rh.daily.canesm5.r1i1p2f1.2014_cog.tif gs://wrf-rh/rh.daily.canesm5.r1i1p2f1.2014.tif
result b''
uploaded future 2015
cp >> : gsutil cp rh.daily.canesm5.r1i1p2f1.2015_cog.tif gs://wrf-rh/rh.daily.canesm5.r1i1p2f1.2015.tif
result b''
uploaded future 2016
cp >> : gsutil cp rh.daily.canesm5.r1i1p2f1.2016_cog.tif gs://wrf-rh/rh.daily.canesm5.r1i1p2f1.2016.tif
result b''
uploaded future 2017
cp >> : gsutil cp rh.daily.canesm5.r1i1p2f1.2017_cog.tif gs://wrf-rh/rh.daily.canesm5.r1i1p2f1.2017.tif
result b''
uploaded future 2018
cp >> : gsutil cp rh.daily.canesm5.r1i1p2f1.2018_cog.tif gs://wrf-rh/rh.daily.canesm5.r1i1p2f1.2018.tif
result b''
uploaded future 2019
cp >> : gsutil cp rh.daily.canesm5.r1i1p2f1.2019_cog.tif gs://wrf-rh/rh.daily.canesm5.r1i1p2f1.2019.tif
result b''
uploaded future 2020
cp >> : gsutil cp rh.daily.canesm5.r1i1p2f1.2020_cog.tif gs://wrf-rh/rh.daily.canesm5.r1i1p2f1.2020.tif
result b''
uploaded future 2021
cp >> : gsutil cp rh.daily.

### upload to ee

In [ ]:
for year in future_years:
    # f_name = f'{wrf_variable_name}.daily.canesm5.r1i1p2f1.{year}'
    f_name = get_saved_file_name(wrf_variable_name, soil_m_level=soil_m_level)
    img = ee.Image.loadGeoTIFF(f'gs://{gcp_bucket}/{f_name}')
    img = img.set(
        'system:time_start', ee.Date(f'{year}-01-01').millis(),
        'system:time_end', ee.Date(f'{year}-12-31').millis(),
        'soil_m_level', soil_m_level,
        'wrf_variable_name', wrf_variable_name,
        'year', year,
    )
    export_image = f'projects/{PROJECT_ID}/assets/{ee_path}/{"_".join(f_name.split("."))}'
    print(f'uploading to {export_image}')
    image_task = ee.batch.Export.image.toAsset(
        image=img,
        description=f'{f_name}',
        assetId=export_image,
        region=img.geometry().bounds(),
        scale=5000,
        maxPixels=1e13,
    )

    image_task.start()


uploading to projects/pyregence-ee/assets/wrf-data/relative-humidity/rh_daily_canesm5_r1i1p2f1_2014_tif
uploading to projects/pyregence-ee/assets/wrf-data/relative-humidity/rh_daily_canesm5_r1i1p2f1_2015_tif
uploading to projects/pyregence-ee/assets/wrf-data/relative-humidity/rh_daily_canesm5_r1i1p2f1_2016_tif
uploading to projects/pyregence-ee/assets/wrf-data/relative-humidity/rh_daily_canesm5_r1i1p2f1_2017_tif
uploading to projects/pyregence-ee/assets/wrf-data/relative-humidity/rh_daily_canesm5_r1i1p2f1_2018_tif
uploading to projects/pyregence-ee/assets/wrf-data/relative-humidity/rh_daily_canesm5_r1i1p2f1_2019_tif
uploading to projects/pyregence-ee/assets/wrf-data/relative-humidity/rh_daily_canesm5_r1i1p2f1_2020_tif
uploading to projects/pyregence-ee/assets/wrf-data/relative-humidity/rh_daily_canesm5_r1i1p2f1_2021_tif
uploading to projects/pyregence-ee/assets/wrf-data/relative-humidity/rh_daily_canesm5_r1i1p2f1_2022_tif
uploading to projects/pyregence-ee/assets/wrf-data/relative-humi